In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Input,GlobalAveragePooling2D
import numpy as np
import os

In [29]:
DATA_DIR="../Dataset/"
TARGET_SIZE=(224,224)
BATCH_SIZE=32
VALIDATION_SPLIT=0.3
EPOCHS=100

In [16]:
train_data_gen = ImageDataGenerator(validation_split=VALIDATION_SPLIT,
                                    rescale=1./255)

train_gen = train_data_gen.flow_from_directory(DATA_DIR,
                                               target_size=TARGET_SIZE,
                                               class_mode='categorical',
                                               shuffle=True,
                                              subset='training')

train_gen = train_data_gen.flow_from_directory(DATA_DIR,
                                               target_size=TARGET_SIZE,
                                               class_mode='categorical',
                                                subset='validation')

Found 7 images belonging to 2 classes.
Found 3 images belonging to 2 classes.


In [28]:
vgg16_model =VGG16(weights = 'imagenet', include_top = False)
x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(28, activation = 'softmax')(x)
model = Model(inputs = vgg16_model.input, outputs = predictions)
for layer in vgg16_model.layers:
    layer.trainable = False
model.compile(optimizer = 'rmsprop',loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#Early Stopping
early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')
#CReating a log file of training
csv= tf.keras.callbacks.CSVLogger('logfile.csv')
#Terminate if Nan Occurs
term_nan=tf.keras.callbacks.TerminateOnNaN() 
#Learning Rate Sceduler
    
history = model.fit_generator(train_gen,epochs=EPOCHS,validation_data=valid_gen,shuffle=True,callbacks=[early,checkpoint,csv,term_nan])